<a href="https://colab.research.google.com/github/VallabhK/TitanicKaggle/blob/main/Titanic_MLfromDisaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import all the required libraries
import pandas as pd
import numpy as np